## Imports

In [1]:
import os
import yaml
import pandas as pd
import s3fs
import matplotlib.pyplot as plt

## Load CSVs from S3

In [2]:
aws_cred = yaml.load(open("../conf/local/credentials.yml"), Loader=yaml.FullLoader)

In [3]:
s3 = s3fs.S3FileSystem(key=aws_cred['dssg']['aws_access_key_id'], secret=aws_cred['dssg']['aws_secret_access_key'])

In [4]:
table_names = ["ESTGESTOR_EMP_OFERTAS.csv",
            "ESTGESTOR_EMP_PEDIDOS.csv",
            "ESTGESTOR_MOV_APRESENTADOS.csv",
            "ESTGESTOR_MOV_CONTROLADOS.csv",
            "ESTGESTOR_MOV_CONVOCADOS.csv",
            "ESTGESTOR_MOV_INTERVENCOES.csv",
            "ESTGESTOR_MOV_OCUPACOES.csv",
            "ESTGESTOR_MOV_REC_INTERNACIONAL.csv",
            "ESTGESTOR_MOV_RMG.csv",
            "ESTGESTOR_MOV_UTENTES.csv",
            "ESTGESTOR_RESP_OFERTAS.csv",
            "ESTUSER_GDE_OFERTAS.csv",
            "ESTUSER_GDE_UTENTES.csv",]

s3_path_template = "s3://iefp-unemployment/SIGAE/{}"

In [5]:
tables = dict()

for name in table_names:
    tables.update({name.strip(".csv"): pd.read_csv(s3_path_template.format(name))})

## Translate Column Names

In [6]:
# Load data dictionary
column_dict = yaml.load(open('../references/SIGAE_dict.yaml'),Loader=yaml.FullLoader)

In [7]:
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from d01_data.translation import translate_columns

In [8]:
dfs_translated = translate_columns(tables.values(), column_dict["SIGAE_DICT"])
tables = dict(zip(table_names, dfs_translated))